<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=500 height=300 align=center class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2024<br>
<font color=3C99D size=5>
    Practical Assignment 3 - Heart Disease Prediction using Hidden Markov Models  <br>
<font color=696880 size=4>
Omid Daliran


# Personal Data

In [1]:
# Set your student number and name
student_number = '401106096'
Name = 'Radin'
Last_Name = 'Shahdaei'

# Libraries

In [2]:
import csv
import numpy as np
import pandas as pd

# Q2: Heart Disease Prediction using Hidden Markov  (100 Points)

# Introduction


In this notebook, we explore the application of Hidden Markov Models (HMM) in predicting heart disease risk based on DNA sequences. Heart disease is a prevalent and life-threatening condition, and early detection is crucial for effective management. By leveraging HMM, we aim to identify regions within DNA sequences associated with high and low GC content, which have been linked to heart disease susceptibility.


# Hidden Markov Models (HMM) (40 points)

Hidden Markov Models are probabilistic models widely used in sequential data analysis, particularly in bioinformatics. They are characterized by states, transitions between states, and emission probabilities associated with each state. In our context, the states represent the underlying biological characteristics of DNA sequences, while the emission probabilities signify the likelihood of observing specific nucleotides given each state.

To facilitate our heart disease classification, we'll implement a custom Hidden Markov Model (HMM) class in Python. This class will include the Viterbi and Forward algorithms.

In [3]:
import numpy as np

class HMM:
    def __init__(self, states: list, emissions: list, start_probabilities: list, transition_probabilities: list, emission_probabilities: list):
        self.states = states
        self.state_labels = {i: state for i, state in enumerate(states)}
        self.emissions = emissions
        self.start_probabilities = start_probabilities
        self.transition_probabilities = transition_probabilities
        self.emission_probabilities = emission_probabilities

    def viterbi(self, sequence: str) -> tuple:
        n_states = len(self.states)
        len_seq = len(sequence)

        dp = np.zeros((n_states, len_seq))
        path = np.zeros((n_states, len_seq), dtype=int)

        for s in range(n_states):
            dp[s, 0] = self.start_probabilities[s] * self.emission_probabilities[s][self.emissions.index(sequence[0])]
            path[s, 0] = s

        for t in range(1, len_seq):
            for s in range(n_states):
                max_prob = dp[0, t-1] * self.transition_probabilities[0][s] * self.emission_probabilities[s][self.emissions.index(sequence[t])]
                max_state = 0
                for s_prime in range(1, n_states):
                    prob = dp[s_prime, t-1] * self.transition_probabilities[s_prime][s] * self.emission_probabilities[s][self.emissions.index(sequence[t])]
                    if prob > max_prob:
                        max_prob = prob
                        max_state = s_prime
                dp[s, t] = max_prob
                path[s, t] = max_state

        max_prob = dp[0, len_seq-1]
        last_state = 0
        for s in range(1, n_states):
            if dp[s, len_seq-1] > max_prob:
                max_prob = dp[s, len_seq-1]
                last_state = s

        opt_path = np.zeros(len_seq, dtype=int)
        opt_path[len_seq-1] = last_state
        for t in range(len_seq-2, -1, -1):
            opt_path[t] = path[opt_path[t+1], t+1]

        state_path = ''.join(self.state_labels[state] for state in opt_path)

        return state_path, max_prob

    def forward_algorithm(self, sequence: str) -> list:
        n_states = len(self.states)
        len_seq = len(sequence)

        forward = np.zeros((n_states, len_seq))

        for s in range(n_states):
            forward[s, 0] = self.start_probabilities[s] * self.emission_probabilities[s][self.emissions.index(sequence[0])]

        for t in range(1, len_seq):
            for s in range(n_states):
                forward[s, t] = sum(forward[s_prime, t-1] * self.transition_probabilities[s_prime][s] for s_prime in range(n_states)) * self.emission_probabilities[s][self.emissions.index(sequence[t])]

        probability_last_states = forward[:, len_seq-1]

        total_probability = sum(probability_last_states)
        if total_probability > 0:
            probability_last_states = probability_last_states / total_probability

        return probability_last_states

GC content, the proportion of guanine (G) and cytosine (C) nucleotides in a DNA sequence, is a fundamental genomic feature. Variations in GC content have been implicated in various biological processes, including gene regulation and disease susceptibility. Specifically, high GC content (H) regions are associated with increased gene expression and potential regulatory elements, while low GC content (L) regions may indicate structural elements or regions prone to mutations. The transition and emission and start probabilities are as shown in the image below:

No Image Available!


In [ ]:
states = ['H', 'L']
emissions = ['A', 'C', 'G', 'T']
start_probabilities = [0.5, 0.5]
transition_probabilities = [
    [0.5, 0.5],  # from H to [H, L]
    [0.4, 0.6]   # from L to [H, L]
]
emission_probabilities = [
    [0.2, 0.3, 0.3, 0.2],  # H state emissions [A, C, G, T]
    [0.3, 0.2, 0.2, 0.3]   # L state emissions [A, C, G, T]
]

hmm = HMM(states, emissions, start_probabilities, transition_probabilities, emission_probabilities)

# Example DNA sequence
sequence = "ATGCGCGATCGATCGAATCGCGT"

# Viterbi algorithm
path, max_probability = hmm.viterbi(sequence)
print("Viterbi Path:", path)
print("Max Probability (Viterbi):", max_probability)

# Forward algorithm
probability = hmm.forward_algorithm(sequence)
print("Probability (Forward Algorithm):", list(probability))


Viterbi Path: LLHHHHHLLHHLLHHLLLHHHHL
Max Probability (Viterbi): 1.1438396227480495e-19
Probability (Forward Algorithm): [0.35818008853498684, 0.6418199114650132]


# Calculating Probabilities for Blood Type, Height, and Weight (10 points)

After obtaining the sequence of H and L regions from the DNA using the Viterbi algorithm, we proceed to calculate the probabilities for other attributes such as blood type, height, weight, and the presence or absence of DCM (DCM, or Dilated Cardiomyopathy, is a heart condition characterized by the enlargement of the heart's left ventricle. It can lead to heart failure if not properly managed.) using the Forward algorithm. In this step, we aim to leverage the Hidden Markov Model (HMM) to infer these attributes based on the observed GC content sequence.


We will use the functionality of the Forward algorithm to compute the probabilities associated with each possible state after observing the sequence.

Attributes and Possible States:
* Blood Type: O, A, B, AB
* Height: 160 cm, 170 cm, 180 cm, 190 cm
* Weight: 50 kg, 65 kg, 80 kg, 95 kg
* DCM: Y (indicating presence) or N (indicating absence)


In [4]:
def probabilities_from_sequence(sequence: str) -> tuple:
    '''
    This function computes the probabilities associated with attributes such as blood type, height, weight,
    and the presence or absence of Dilated Cardiomyopathy (DCM) based on an input DNA sequence.

    ### Input:
    - `sequence`: The observed DNA sequence for which probabilities are to be computed.

    ### Output:
    The function returns dictionaries containing the probabilities for each possible state of the respective attributes:
    - `height_probability_dic`: Probabilities for height categories (160 cm, 170 cm, 180 cm, 190 cm).
    - `weight_probability_dic`: Probabilities for weight categories (50 kg, 65 kg, 80 kg, 95 kg).
    - `blood_type_probability_dic`: Probabilities for blood types (O, A, B, AB).
    - `DCM_probability_dic`: Probabilities for the presence (Y) or absence (N) of Dilated Cardiomyopathy (DCM).
    '''
    # Define the emission probabilities for each feature
    height_emissions = [{'H': 0.2, 'L': 0.8}, {'H': 0.3, 'L': 0.7},
                        {'H': 0.9, 'L': 0.1}, {'H': 0.8, 'L': 0.2}]

    weight_emissions = [{'H': 0.3, 'L': 0.7}, {'H': 0.1, 'L': 0.9},
                        {'H': 0.2, 'L': 0.8}, {'H': 0.7, 'L': 0.3}]

    blood_type_emissions = [{'H': 0.75, 'L': 0.25}, {'H': 0.8, 'L': 0.2},
                            {'H': 0.85, 'L': 0.15}, {'H': 0.7, 'L': 0.3}]

    DCM_emissions = [{'H': 0.7, 'L': 0.3}, {'H': 0.6, 'L': 0.4}]

    # Define the transition probabilities for each feature
    height_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
    weight_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
    blood_type_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
    DCM_transition_probabilities = [[0.9, 0.1], [0.1, 0.9]]

    # Define start probabilities for each state
    height_start_probabilities = [0.1, 0.4, 0.4, 0.1]
    weight_start_probabilities = [0.15, 0.35, 0.35, 0.15]
    blood_type_start_probabilities = [0.1, 0.35, 0.35, 0.2]
    DCM_start_probabilities = [0.25, 0.75]

    def forward_algorithm(emissions, transitions, start_probabilities):
        N = len(emissions)  # Number of states
        T = len(sequence)   # Length of the sequence

        forward = [[0.0] * N for _ in range(T)]

        for i in range(N):
            forward[0][i] = start_probabilities[i] * emissions[i][sequence[0]]

        for t in range(1, T):
            for j in range(N):
                forward[t][j] = sum(forward[t-1][i] * transitions[i][j] * emissions[j][sequence[t]] for i in range(N))

        # Termination step
        probabilities = [sum(forward[t][i] for i in range(N)) for t in range(T)]
        final_probabilities = [forward[T-1][i] / sum(forward[T-1]) for i in range(N)]

        return final_probabilities

    height_probability = forward_algorithm(height_emissions, height_transition_probabilities, height_start_probabilities)
    weight_probability = forward_algorithm(weight_emissions, weight_transition_probabilities, weight_start_probabilities)
    blood_type_probability = forward_algorithm(blood_type_emissions, blood_type_transition_probabilities, blood_type_start_probabilities)
    DCM_probability = forward_algorithm(DCM_emissions, DCM_transition_probabilities, DCM_start_probabilities)

    height_states = [160, 170, 180, 190]
    weight_states = [50, 65, 80, 95]
    blood_type_states = ['O', 'A', 'B', 'AB']
    DCM_states = ['Y', 'N']

    # Create dictionaries for each attribute
    height_dict = {height_states[i]: height_probability[i] for i in range(len(height_states))}
    weight_dict = {weight_states[i]: weight_probability[i] for i in range(len(weight_states))}
    blood_type_dict = {blood_type_states[i]: blood_type_probability[i] for i in range(len(blood_type_states))}
    DCM_dict = {DCM_states[i]: DCM_probability[i] for i in range(len(DCM_states))}

    return height_dict, weight_dict, blood_type_dict, DCM_dict


In [ ]:
# Run this code to check your implementation
# Example sequence of 'H' and 'L'
sequence = "HHLLHHHLLHLLLLHHLL"

height_dict, weight_dict, blood_type_dict, DCM_dict = probabilities_from_sequence(sequence)

print("Probability of height given sequence:", height_dict)
print("Probability of weight given sequence:", weight_dict)
print("Probability of blood type given sequence:", blood_type_dict)
print("Probability of DCM given sequence:", DCM_dict)

Probability of height given sequence: {160: 0.36605931748711135, 170: 0.5760951509787055, 180: 0.015850353367627773, 190: 0.04199517816655549}
Probability of weight given sequence: {50: 0.2845952908892867, 65: 0.42707075390249144, 80: 0.20957356322963908, 95: 0.07876039197858277}
Probability of blood type given sequence: {'O': 0.29628284733090365, 'A': 0.3305080573190477, 'B': 0.08789883086305705, 'AB': 0.2853102644869917}
Probability of DCM given sequence: {'Y': 0.3297758181008331, 'N': 0.6702241818991669}


# Bayesian Network for Heart Disease Probability Calculation (30 points)

In this section, we introduce a Bayesian network to calculate the probability of heart disease based on various features. We assume that the relationships between features follow a specific structure, depicted in the image below.

No Image Available!

The Bayesian network comprises nodes representing different features, including sex, weight, height, Dilated Cardiomyopathy (DCM), blood type, and heart disease. The directed edges between nodes indicate probabilistic dependencies between them.

Function Implementation:
We will implement two functions to facilitate the calculation of heart disease probability:

1. calculate_sex_probability:

  This function calculates the probability of sex based on the given probabilities of weight and height.

2. calculate_heart_disease_probability:

  This function computes the probability of heart disease based on the probabilities of sex, DCM, and blood type.

In [10]:
# The probability of sex given height and weight, the tuple is in this order: (sex, height, weight)
P_sex_given_hw = {
    ('M', 160, 50): 0.7, ('F', 160, 50): 0.3,
    ('M', 160, 65): 0.2, ('F', 160, 65): 0.8,
    ('M', 160, 80): 0.3, ('F', 160, 80): 0.7,
    ('M', 160, 95): 0.2, ('F', 160, 95): 0.8,
    ('M', 170, 50): 0.8, ('F', 170, 50): 0.2,
    ('M', 170, 65): 0.75, ('F', 170, 65): 0.25,
    ('M', 170, 80): 0.4, ('F', 170, 80): 0.6,
    ('M', 170, 95): 0.3, ('F', 170, 95): 0.7,
    ('M', 180, 50): 0.9, ('F', 180, 50): 0.1,
    ('M', 180, 65): 0.7, ('F', 180, 65): 0.3,
    ('M', 180, 80): 0.65, ('F', 180, 80): 0.35,
    ('M', 180, 95): 0.4, ('F', 180, 95): 0.6,
    ('M', 190, 50): 0.95, ('F', 190, 50): 0.05,
    ('M', 190, 65): 0.8, ('F', 190, 65): 0.2,
    ('M', 190, 80): 0.6, ('F', 190, 80): 0.4,
    ('M', 190, 95): 0.95, ('F', 190, 95): 0.05
}

In [11]:
import numpy as np

def calculate_sex_probability(height_prob: dict, weight_prob: dict) -> dict:
    '''
    This function calculates the probability of sex based on the probabilities of height and weight.
    It assumes that the probabilities of height and weight are independent and uses them to compute the combined
    probability distribution for each sex ('M' and 'F').

    Parameters:
    - height_prob: Dictionary containing probabilities for height categories.
    - weight_prob: Dictionary containing probabilities for weight categories.

    Returns:
    - Dictionary containing probabilities for male (M) and female (F) sexes.
    '''

    # Initialize the probabilities for each sex
    sex_probability = {'M': 0, 'F': 0}

    # Compute combined probability for each sex
    for (sex, height, weight), prob in P_sex_given_hw.items():
        if height in height_prob and weight in weight_prob:
            combined_prob = height_prob[height] * weight_prob[weight] * prob
            sex_probability[sex] += combined_prob

    # Normalize the probabilities to sum to 1
    total_prob = sex_probability['M'] + sex_probability['F']
    if total_prob > 0:
        sex_probability['M'] /= total_prob
        sex_probability['F'] /= total_prob

    return sex_probability

# Example height and weight probabilities
P_height = {160: 0.25, 170: 0.25, 180: 0.25, 190: 0.25}
P_weight = {50: 0.25, 65: 0.25, 80: 0.25, 95: 0.25}

# Calculate sex probabilities
sex_probabilities = calculate_sex_probability(P_height, P_weight)
print("Probability of sex:", sex_probabilities)


Probability of sex: {'M': 0.6, 'F': 0.39999999999999997}


In [ ]:
# Run this code to test you implementation
# Example probabilities of height
P_height = {160: 0.1, 170: 0.3, 180: 0.4, 190: 0.2}
# Example probabilities of weight
P_weight = {50: 0.2, 65: 0.3, 80: 0.4, 95: 0.1}
# Example conditional probabilities of sex given height and weight


sex_probabilities = calculate_sex_probability(P_height, P_weight)
print("Probability of sex:", sex_probabilities)

Probability of sex: {'M': 0.6355000000000001, 'F': 0.3645}


In [13]:
# The probability of hear disease given sex, DCM and blood type, the tuple is in this order: (heart disease, sex, blood type, DCM)
P_heart_disease_given_parents = {
    ('N', 'M', 'O', 'Y'): 0.3, ('Y', 'M', 'O', 'Y'): 0.7,
    ('N', 'M', 'A', 'Y'): 0.4, ('Y', 'M', 'A', 'Y'): 0.6,
    ('N', 'M', 'B', 'Y'): 0.5, ('Y', 'M', 'B', 'Y'): 0.5,
    ('N', 'M', 'AB', 'Y'): 0.6, ('Y', 'M', 'AB', 'Y'): 0.4,
    ('N', 'F', 'O', 'Y'): 0.6, ('Y', 'F', 'O', 'Y'): 0.4,
    ('N', 'F', 'A', 'Y'): 0.3, ('Y', 'F', 'A', 'Y'): 0.7,
    ('N', 'F', 'B', 'Y'): 0.4, ('Y', 'F', 'B', 'Y'): 0.6,
    ('N', 'F', 'AB', 'Y'): 0.7, ('Y', 'F', 'AB', 'Y'): 0.3,
    ('N', 'M', 'O', 'N'): 0.6, ('Y', 'M', 'O', 'N'): 0.4,
    ('N', 'M', 'A', 'N'): 0.15, ('Y', 'M', 'A', 'N'): 0.85,
    ('N', 'M', 'B', 'N'): 0.4, ('Y', 'M', 'B', 'N'): 0.6,
    ('N', 'M', 'AB', 'N'): 0.7, ('Y', 'M', 'AB', 'N'): 0.3,
    ('N', 'F', 'O', 'N'): 0.8, ('Y', 'F', 'O', 'N'): 0.2,
    ('N', 'F', 'A', 'N'): 0.7, ('Y', 'F', 'A', 'N'): 0.3,
    ('N', 'F', 'B', 'N'): 0.65, ('Y', 'F', 'B', 'N'): 0.35,
    ('N', 'F', 'AB', 'N'): 0.35, ('Y', 'F', 'AB', 'N'): 0.65,
}

In [14]:
import numpy as np

def calculate_heart_disease_probability(DCM_prob: dict, sex_prob: dict, blood_type_prob: dict) -> dict:
    '''
    Function: Calculate Heart Disease Probability

    This function calculates the probability of heart disease based on the probabilities of Dilated Cardiomyopathy (DCM),
    sex, and blood type. It uses a dictionary of conditional probabilities for heart disease given DCM, sex, and blood type.

    Parameters:
    - DCM_prob: Dictionary containing probabilities for DCM categories ('Y' and 'N').
    - sex_prob: Dictionary containing probabilities for sex categories ('M' and 'F').
    - blood_type_prob: Dictionary containing probabilities for blood type categories ('O', 'A', 'B', 'AB').

    Returns:
    - Dictionary containing probabilities for the presence ('Y') and absence ('N') of heart disease.
    '''
    # Initialize the probabilities for heart disease
    heart_disease_probability = {'Y': 0, 'N': 0}

    # Calculate the probability of heart disease
    for (hd, sex, bt, dcm), p_hd in P_heart_disease_given_parents.items():
        if sex in sex_prob and bt in blood_type_prob and dcm in DCM_prob:
            # Calculate the joint probability
            joint_prob = sex_prob[sex] * blood_type_prob[bt] * DCM_prob[dcm] * p_hd
            # Accumulate the probabilities
            heart_disease_probability[hd] += joint_prob

    # Normalize the probabilities to sum to 1
    total_prob = heart_disease_probability['Y'] + heart_disease_probability['N']
    if total_prob > 0:
        heart_disease_probability['Y'] /= total_prob
        heart_disease_probability['N'] /= total_prob

    return heart_disease_probability


In [ ]:
# Run this code to test you implementation
# Example probabilities of DCM
P_DCM = {'Y': 0.2, 'N': 0.8}
# Example probabilities of blood type
P_blood_type = {'O': 0.4, 'A': 0.3, 'B': 0.2, 'AB': 0.1}
# Example probabilities of sex
P_sex = {'M': 0.3, 'F': 0.7}


heart_disease_probabilities = calculate_heart_disease_probability(P_DCM, P_sex, P_blood_type)
print("Probability of heart disease: ",heart_disease_probabilities)

Probability of heart disease:  {'Y': 0.41519999999999996, 'N': 0.5848}


# Loading DNA Sequences and Heart Disease Probability Calculation (20 points)

In this part, we will load the DNA sequences from the file DNA_patients.csv and calculate the probability of heart disease for each patient. Subsequently, we will classify the patients based on their heart disease probability and save the data.

In [15]:
# Read the CSV file into a DataFrame
df = pd.read_csv('DNA_patients.csv')

df.head()

,ID,DNA
0,1,CTCCAATACCCCCCACAAGAACACACCCATAAAATTGCAACCCACA...
1,2,TTGATGTAGAAGTATATTTGTTGGGTATTTGAGGTAACGTTATTAG...
2,3,ATATAATTTAAAGTCACTGGAAAAAAACAACCTAATAAAAACCACC...
3,4,TAAAGACAAAATTAAATTGAAGTAATGTTATGTTAAAATTTTGAAT...
4,5,AGCGTTTGTTCGTTAGCCGTAGGCAATGACGTGATTCAGGTCTGTG...


In [16]:
# TODO: complete the code
# Extract DNA sequences and store them in a list
DNAs = df['DNA']

states = ['H', 'L']
emissions = ['A', 'C', 'G', 'T']
start_probabilities = [0.5, 0.5]
transition_probabilities = [
    [0.5, 0.5],  # from H to [H, L]
    [0.4, 0.6]   # from L to [H, L]
]
emission_probabilities = [
    [0.2, 0.3, 0.3, 0.2],  # H state emissions [A, C, G, T]
    [0.3, 0.2, 0.2, 0.3]   # L state emissions [A, C, G, T]
]

hmm = HMM(states, emissions, start_probabilities, transition_probabilities, emission_probabilities)

GC_content_seqs = []

for seq in DNAs:
  path, max_probability = hmm.viterbi(seq)
  GC_content_seqs.append(path)

In [17]:
# Assuming we have already computed the dictionaries DCM_prob, sex_prob, and blood_type_prob

heart_disease_probabilities = []

for seq in GC_content_seqs:
    height_prob, weight_prob, blood_type_prob, DCM_prob = probabilities_from_sequence(seq)
    sex_probabilities = calculate_sex_probability(height_prob, weight_prob)
    hd_prob = calculate_heart_disease_probability(DCM_prob, sex_probabilities, blood_type_prob)
    heart_disease_probabilities.append(hd_prob)

heart_disease_classified = []

for prob in heart_disease_probabilities:
    if prob['Y'] > 0.5:
        heart_disease_classified.append(1)
    else:
        heart_disease_classified.append(0)




In [18]:
# Write the result to CSV
labels = [(i+1, heart_disease_classified[i]) for i in range(len(heart_disease_classified))]
csv_filename = "heart_disease_result.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Label'])
    for id, label in labels:
        writer.writerow([id, label])

**Note: You should upload heart_disease_result.csv along side your notebook.**